# KEGG BRITE functional hierarchies
In this notebook, a mapping procedure is carried out with the purpose of creating a final table that links the hierarchical functional information contained in KEGG BRITE database to the genes present in the PanCancer gene-expression dataset.

In [1]:
import pandas as pd
import numpy as np

##  KEGG and PanCancer data loading

- Data coming from KEGG database is fetched on `KEGG_BRITE_Data_Obtaining` notebook.
- Data coming from PanCancer dataset is obtained from `PanCancer_Gene_Filtering` notebook.

In [2]:
%%time
# KEGG information
hsa = pd.read_csv('data/KEGG/KEGG_gene_hsa.csv', sep='\t', 
                  engine='python', header=None, index_col=None, names=['keggId', 'geneName'])
hsa_brite = pd.read_csv('data/KEGG/KEGG_hsa_brite.csv', 
                        sep='\t', engine='python', header=None, index_col=None, names=['keggBriteId', 'keggId'])
brite = pd.read_csv('data/KEGG/KEGG_brite.csv', 
                    engine='python', header=0, index_col=0)

# PanCancer gene-expression dataset
pancan_ex = pd.read_hdf("data/PanCancer/mad_filter_pancan_all_TCGA_20.h5", key = "expression")

CPU times: user 2.28 s, sys: 839 ms, total: 3.12 s
Wall time: 3.18 s


Get ENSEMBL IDs:

In [3]:
ens_genes = pancan_ex.columns

In [4]:
print("Initial number of genes: {}".format(len(ens_genes)))

Initial number of genes: 20000


In [5]:
print("Total number of samples (patients): {}".format(pancan_ex.shape[0]))

Total number of samples (patients): 10535


## MAPPING: From Ensembl Gene ID to KEGG BRITE functional hierarchy
KEGG relates Hugo-gene-names to an internal id (KEGG gene id) that is useful to find out the corresponding KEGG BRITE ids (hierarchical functional annotations) of each gene. Because of genes in PanCancer dataset are indexed by ENSEMBL-ids, a mapping procedure needs to be performed, that associates ENSEMBL-id to Hugo-gene-name, Hugo-gene-name to KEGG gene id and KEGG gene id to KEGG BRITE id.

### Mapping: Ensembl Gene ID to  Hugo Gene Name

We load a table relating ENSEMBL-ids and Hugo-gene-names, downloaded from: https://raw.githubusercontent.com/jvivian/docker_tools/master/gencode_hugo_mapping/attrs.tsv

In [6]:
%%time
hugo_ens = pd.read_csv('data/hugo_ensembl.tsv', 
                       sep='\t', engine='python', index_col=None)

CPU times: user 1.07 s, sys: 58.7 ms, total: 1.13 s
Wall time: 1.14 s


In [7]:
hugo_ens = hugo_ens[~hugo_ens.duplicated(subset=['geneId', 'geneName'])]
hugo_ens.shape

(65670, 13)

We only select genes contained in the expression datasets:

In [8]:
hugo_ens = hugo_ens[hugo_ens['geneId'].isin(ens_genes)]
hugo_ens.shape

(20000, 13)

In [9]:
gene_mapping = hugo_ens[['geneId', 'geneName']]

In [10]:
gene_mapping.head()

geneId       geneName
2   ENSG00000227232.5         WASH7P
12  ENSG00000238009.6   RP11-34P13.7
13  ENSG00000239945.1   RP11-34P13.8
18  ENSG00000233750.3         CICP27
19  ENSG00000268903.1  RP11-34P13.15

In [11]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [12]:
print("Number of Hugo genes contained in the expression datasets:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets: 19959


In [13]:
print("Number of ENSEMBLE genes contained in the expression datasets:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets: 20000


### Mapping: Hugo Gene Name to KEGG gene id

In [14]:
# Check if we can split using the next two characters
sum(map(lambda x: ',' in x or ';' in x, gene_mapping.geneName))

0

In [15]:
%%time
import itertools

# We create a dataframe from a list with all (keggId, hugoGeneName) pairs contained in hsa dataframe
hsa_hugo = pd.DataFrame(data=list(itertools.chain.from_iterable(hsa.apply(
    lambda x: list(zip(itertools.repeat(x[0]), x[1].replace(';', ',').split(', '))), axis=1))), 
                        columns=['keggId', 'geneName'])

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.05 s


In [16]:
print(hsa_hugo.shape)
hsa_hugo.head()

(126869, 2)


keggId geneName
0  hsa:4549     RNR1
1  hsa:4549   MTRNR1
2  hsa:4549  MT-RNR1
3  hsa:4549   s-rRNA
4  hsa:4550     RNR2

In [17]:
# Sanity check
sum(hsa_hugo.duplicated())

86

In [18]:
# Remove duplicated rows
hsa_hugo = hsa_hugo[~hsa_hugo.duplicated()]

In [19]:
gene_mapping = pd.merge(gene_mapping, hsa_hugo, on='geneName')

In [20]:
gene_mapping.head()

geneId   geneName      keggId
0  ENSG00000227232.5     WASH7P  hsa:653635
1  ENSG00000226210.3     WASH7P  hsa:653635
2  ENSG00000177757.2     FAM87B  hsa:400728
3  ENSG00000225880.5  LINC00115   hsa:79854
4  ENSG00000230368.2     FAM41C  hsa:284593

In [21]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [22]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG IDs:", 
      len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG IDs: 12475


In [23]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG IDs:", 
      len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG IDs: 12506


### Mapping: KEGG gene id to KEGG BRITE id

In [24]:
gene_mapping = pd.merge(gene_mapping, hsa_brite, on='keggId')

In [25]:
gene_mapping.head()

geneId geneName      keggId  keggBriteId
0  ENSG00000187961.13   KLHL17  hsa:339451  br:hsa00001
1  ENSG00000187961.13   KLHL17  hsa:339451  br:hsa04121
2  ENSG00000188290.10     HES4   hsa:57801  br:hsa00001
3  ENSG00000188290.10     HES4   hsa:57801  br:hsa03000
4   ENSG00000187608.8    ISG15    hsa:9636  br:hsa00001

In [26]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [27]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE" +
      " IDs:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE IDs: 7495


In [28]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE" +
      " IDs:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE IDs: 7509


### Mapping: KEGG BRITE id to KEGG BRITE functional hierarchies

We pre-process the KEGG Brite IDs contained in `gene_mapping` dataset, in order to match the KEGG Brite IDs contained in `brite` dataset:

In [29]:
# Check all KEGG Brite IDs contained in gene_mapping dataframe have the same length
id_len = len(gene_mapping['keggBriteId'][0])
all(gene_mapping['keggBriteId'].apply(lambda x: len(x) == id_len))

True

In [30]:
# Check all KEGG Brite IDs contained in brite dataframe have the same length
brite_id_len = len(brite['keggBriteId'][0])
all(brite['keggBriteId'].apply(lambda x: len(x.split('_')[0]) == brite_id_len))

True

In [31]:
# Extract the last four digits of each KEGG Brite ID
gene_mapping['keggBriteId'] = gene_mapping['keggBriteId'].apply(lambda x: x[-4:])

In [32]:
hsa_brite['keggBriteId'].apply(lambda x: x[-4:]).drop_duplicates()

0        0001
13910    1000
17827    0199
17888    0535
17941    0536
18143    1001
18667    3000
19802    1002
20352    1003
20576    1004
20655    1006
20670    1007
20728    1009
21185    3110
21408    4121
22225    2044
22241    4054
22301    2042
22307    2048
22310    3011
22497    3012
22590    3041
22948    3021
23194    4131
24566    3100
24880    2000
25546    3051
25621    3029
25987    3009
26232    3036
27423    3310
27471    0537
27556    4812
28064    3019
28505    3032
28630    4052
28867    3400
29186    4090
29628    4040
29938    4515
30130    4147
31292    4091
31428    4031
31620    3016
31783    4030
32600    4050
Name: keggBriteId, dtype: object

Now, we perform the final mapping:

In [33]:
gene_mapping = pd.merge(gene_mapping, brite, on='keggBriteId')

In [34]:
gene_mapping.head()

geneId  geneName      keggId keggBriteId  \
0  ENSG00000187961.13    KLHL17  hsa:339451        0001   
1  ENSG00000188290.10      HES4   hsa:57801        0001   
2   ENSG00000187608.8     ISG15    hsa:9636        0001   
3  ENSG00000188157.13      AGRN  hsa:375790        0001   
4  ENSG00000186891.13  TNFRSF18    hsa:8784        0001   

  Functional Annotation Group Functional Annotation Subgroup  \
0          Genes and Proteins          Orthologs and modules   
1          Genes and Proteins          Orthologs and modules   
2          Genes and Proteins          Orthologs and modules   
3          Genes and Proteins          Orthologs and modules   
4          Genes and Proteins          Orthologs and modules   

  Functional Annotation  
0   KEGG Orthology (KO)  
1   KEGG Orthology (KO)  
2   KEGG Orthology (KO)  
3   KEGG Orthology (KO)  
4   KEGG Orthology (KO)

In [35]:
# Sanity check
sum(gene_mapping.duplicated())

0

In [36]:
print("Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE" +
      " functional annotations:", len(set(gene_mapping['geneName'])))

Number of Hugo genes contained in the expression datasets that are associated with KEGG BRITE functional annotations: 7495


In [37]:
print("Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE" +
      " functional annotations:", len(set(gene_mapping['geneId'])))

Number of ENSEMBLE genes contained in the expression datasets that are associated with KEGG BRITE functional annotations: 7509


In [38]:
gene_mapping['Functional Annotation Group'].value_counts()

Genes and Proteins    18494
Name: Functional Annotation Group, dtype: int64

In [39]:
gene_mapping['Functional Annotation Subgroup'].value_counts()

Orthologs and modules                                 7938
Protein families: signaling and cellular processes    3715
Protein families: metabolism                          3619
Protein families: genetic information processing      3209
RNA family                                              13
Name: Functional Annotation Subgroup, dtype: int64

In [40]:
gene_mapping['Functional Annotation'].value_counts()

KEGG Orthology (KO)                                     7938
Enzymes                                                 2302
Membrane trafficking                                     770
Transcription factors                                    754
Exosome                                                  659
Chromosome and associated proteins                       611
Transporters                                             486
CD molecules                                             411
Cytoskeleton proteins                                    379
Peptidases                                               369
Protein kinases                                          365
G protein-coupled receptors                              343
Ubiquitin system                                         324
Protein phosphatases and associated proteins             275
Ion channels                                             265
Cell adhesion molecules                                  205
Cytokines and growth fac

## KEGG BRITE functional hierarchies dataset

Lastly, we create a final table that links the hierarchical functional information from KEGG BRITE to the genes present in the PanCancer gene-expression dataset.

In [42]:
%%time
exp_to_tree_map = pancan_ex.T
exp_to_tree_map['geneId'] = exp_to_tree_map.index
exp_to_tree_map['tamPixel'] = np.ones(exp_to_tree_map.shape[0])
exp_to_tree_map['order'] = pancan_ex.mean(axis=0) # sort genes by mean expression values 
exp_to_tree_map = pd.merge(gene_mapping, exp_to_tree_map, on='geneId')

CPU times: user 3.88 s, sys: 2 s, total: 5.87 s
Wall time: 4.21 s


In [43]:
exp_to_tree_map.head()

geneId geneName      keggId keggBriteId  \
0  ENSG00000187961.13   KLHL17  hsa:339451        0001   
1  ENSG00000187961.13   KLHL17  hsa:339451        4121   
2  ENSG00000188290.10     HES4   hsa:57801        0001   
3  ENSG00000188290.10     HES4   hsa:57801        3000   
4   ENSG00000187608.8    ISG15    hsa:9636        0001   

  Functional Annotation Group  \
0          Genes and Proteins   
1          Genes and Proteins   
2          Genes and Proteins   
3          Genes and Proteins   
4          Genes and Proteins   

                     Functional Annotation Subgroup  Functional Annotation  \
0                             Orthologs and modules    KEGG Orthology (KO)   
1  Protein families: genetic information processing       Ubiquitin system   
2                             Orthologs and modules    KEGG Orthology (KO)   
3  Protein families: genetic information processing  Transcription factors   
4                             Orthologs and modules    KEGG Orthology (KO)   

   TCGA-02-0047-01  TCGA-02-0055-01  TCGA-02-2483-01  ...  TCGA-ZS-A9CE-01  \
0           1.3225           2.3135           2.5707  ...           0.8883   
1           1.3225           2.3135           2.5707  ...           0.8883   
2           4.1604           3.6148           3.8729  ...          -0.3201   
3           4.1604           3.6148           3.8729  ...          -0.3201   
4           5.8166           6.9599           5.9072  ...           6.4074   

   TCGA-ZS-A9CF-01  TCGA-ZS-A9CF-02  TCGA-ZS-A9CG-01  TCGA-ZT-A8OM-01  \
0           1.7532           0.7664           0.2998           1.9226   
1           1.7532           0.7664           0.2998           1.9226   
2           0.9642          -0.8084           0.2762           5.8595   
3           0.9642          -0.8084           0.2762           5.8595   
4           6.1183           5.0592           5.2122           7.4520   

   TCGA-ZU-A8S4-01  TCGA-ZU-A8S4-11  TCGA-ZX-AA5X-01  tamPixel     order  
0           2.7336          -0.3752           2.7972       1.0  1.992469  
1           2.7336          -0.3752           2.7972       1.0  1.992469  
2           3.9156          -0.1504           4.3751       1.0  2.791702  
3           3.9156          -0.1504           4.3751       1.0  2.791702  
4           6.6671           4.7783           9.7576       1.0  6.491173  

[5 rows x 10544 columns]

In [44]:
%%time
# Sanity check
sum(exp_to_tree_map.duplicated())

CPU times: user 4.92 s, sys: 288 ms, total: 5.21 s
Wall time: 5.21 s


0

In [45]:
# Sanity check (number of ENSEMBLE genes contained in the final dataset)
len(set(exp_to_tree_map['geneId']))

7509

In [46]:
exp_to_tree_map.shape

(18494, 10544)

Finally, we save the final dataset, which is used as input data in `2-KEGG_BRITE_Treemap` notebook. This R notebook implements the generation of the final gene-expression treemap images.

In [47]:
%%time
# Save dataset
exp_to_tree_map.to_csv("./KEGG_exp_to_tree_map.csv", index=False)

CPU times: user 2min 8s, sys: 569 ms, total: 2min 9s
Wall time: 2min 10s
